In [ ]:
# ARC-AGI Task Visualization

This notebook visualizes ARC-AGI puzzles from training dataset JSON files. It displays input-output pairs in a clear format to help understand the dataset structure and patterns.

## Features:
- Load any ARC training JSON file by path
- Visualize all training pairs in 2-column format (Input | Output)
- Handle variable number of puzzles per file
- Compatible with Google Colab

In [ ]:
# Setup for Google Colab
!git clone https://github.com/CalebTalley2024/ARC-AGI-2.git
%cd ARC-AGI-2

!git checkout vedant

# Install exact package versions for reproducibility
!pip install --quiet \
    numpy==1.24.4 \
    matplotlib==3.7.5 \
    pandas==2.0.3 \
    scipy==1.10.1 \
    scikit-learn==1.3.2 \
    torch==2.2.2 \
    torchvision==0.17.2 \
    torchaudio==2.2.2 \
    transformers==4.46.3 \
    huggingface-hub==0.36.0 \
    seaborn==0.13.2 \
    plotly==6.4.0 \
    tqdm==4.67.1 \
    pyyaml==6.0.3 \
    requests==2.32.4 \
    packaging==25.0 \
    jsonschema==4.23.0 \
    fastjsonschema==2.21.2 \
    jinja2==3.1.6 \
    markupsafe==2.1.5 \
    urllib3==2.2.3 \
    certifi==2025.10.5 \
    charset-normalizer==3.4.4 \
    idna==3.11 \
    python-dateutil==2.9.0.post0 \
    pytz==2025.2 \
    tzdata==2025.2 \
    six==1.17.0 \
    setuptools==75.3.2

# Install the package in development mode
!pip install -e .

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os

# Color map for visualization (0-9 colors for ARC grids)
COLOR_MAP = [
    '#000000',  # 0: black
    '#0074D9',  # 1: blue
    '#FF4136',  # 2: red  
    '#2ECC40',  # 3: green
    '#FFDC00',  # 4: yellow
    '#AAAAAA',  # 5: grey
    '#F012BE',  # 6: magenta
    '#FF851B',  # 7: orange
    '#7FDBFF',  # 8: sky blue
    '#870C25'   # 9: brown
]

print("Libraries loaded successfully!")

In [ ]:
def load_arc_data(txt_file_path):
    """
    Load ARC training or evaluation data from txt file containing task IDs
    
    Args:
        txt_file_path: Path to .txt file containing list of task IDs
        
    Returns:
        Dictionary with task_id as keys and task data as values
    """
    try:
        # Read the txt file to get task IDs
        with open(txt_file_path, 'r') as f:
            task_ids = [line.strip() for line in f if line.strip()]
        
        # Determine the JSON directory based on txt file name
        txt_path = Path(txt_file_path)
        base_dir = txt_path.parent
        
        if 'training' in txt_path.name:
            json_dir = base_dir / 'training'
        elif 'evaluation' in txt_path.name:
            json_dir = base_dir / 'evaluation'
        else:
            print(f"Unknown data type from file name: {txt_path.name}")
            return None
        
        if not json_dir.exists():
            print(f"JSON directory not found: {json_dir}")
            return None
        
        # Load all JSON files for the task IDs
        data = {}
        loaded_count = 0
        missing_files = []
        
        for task_id in task_ids:
            json_file = json_dir / f"{task_id}.json"
            
            if json_file.exists():
                try:
                    with open(json_file, 'r') as f:
                        task_data = json.load(f)
                    data[task_id] = task_data
                    loaded_count += 1
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON file {json_file}: {e}")
            else:
                missing_files.append(task_id)
        
        print(f"Loaded {loaded_count} tasks from {txt_file_path}")
        print(f"JSON files source: {json_dir}")
        
        if missing_files:
            print(f"Missing JSON files for {len(missing_files)} task IDs:")
            for task_id in missing_files[:5]:  # Show first 5 missing
                print(f"   {task_id}.json")
            if len(missing_files) > 5:
                print(f"   ... and {len(missing_files) - 5} more")
        
        return data
        
    except FileNotFoundError:
        print(f"Data file not found: {txt_file_path}")
        return None
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

def load_random_tasks(txt_file_path, num_tasks=3):
    """
    Load random subset of tasks from txt file containing task IDs
    
    Args:
        txt_file_path: Path to .txt file containing list of task IDs
        num_tasks: Number of random tasks to load
        
    Returns:
        Dictionary with task_id as keys and task data as values
    """
    try:
        # Read the txt file to get task IDs
        with open(txt_file_path, 'r') as f:
            all_task_ids = [line.strip() for line in f if line.strip()]
        
        # Randomly select subset of task IDs
        import random
        if len(all_task_ids) <= num_tasks:
            selected_task_ids = all_task_ids
            print(f"Selected all {len(all_task_ids)} available tasks")
        else:
            selected_task_ids = random.sample(all_task_ids, num_tasks)
            print(f"Randomly selected {num_tasks} tasks from {len(all_task_ids)} available")
        
        # Determine the JSON directory based on txt file name
        txt_path = Path(txt_file_path)
        base_dir = txt_path.parent
        
        if 'training' in txt_path.name:
            json_dir = base_dir / 'training'
        elif 'evaluation' in txt_path.name:
            json_dir = base_dir / 'evaluation'
        else:
            print(f"Unknown data type from file name: {txt_path.name}")
            return None
        
        if not json_dir.exists():
            print(f"JSON directory not found: {json_dir}")
            return None
        
        # Load JSON files for selected task IDs
        data = {}
        loaded_count = 0
        missing_files = []
        
        for task_id in selected_task_ids:
            json_file = json_dir / f"{task_id}.json"
            
            if json_file.exists():
                try:
                    with open(json_file, 'r') as f:
                        task_data = json.load(f)
                    data[task_id] = task_data
                    loaded_count += 1
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON file {json_file}: {e}")
            else:
                missing_files.append(task_id)
        
        print(f"Successfully loaded {loaded_count} JSON files")
        if missing_files:
            print(f"Missing JSON files: {missing_files}")
        
        return data
        
    except FileNotFoundError:
        print(f"Data file not found: {txt_file_path}")
        return None
    except Exception as e:
        print(f"Error loading random tasks: {e}")
        return None

def plot_grid(grid, title="", ax=None):
    """Plot a single ARC grid with proper colors"""
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    
    grid_array = np.array(grid)
    height, width = grid_array.shape
    
    # Create color mapping
    colored_grid = np.zeros((height, width, 3))
    for i in range(height):
        for j in range(width):
            color_hex = COLOR_MAP[grid_array[i, j]]
            # Convert hex to RGB
            r = int(color_hex[1:3], 16) / 255.0
            g = int(color_hex[3:5], 16) / 255.0
            b = int(color_hex[5:7], 16) / 255.0
            colored_grid[i, j] = [r, g, b]
    
    ax.imshow(colored_grid, interpolation='nearest')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Add grid lines
    for i in range(height + 1):
        ax.axhline(i - 0.5, color='white', linewidth=1)
    for j in range(width + 1):
        ax.axvline(j - 0.5, color='white', linewidth=1)
    
    return ax

print("Data loading functions defined!")
print("Usage:")
print("  load_arc_data('data/raw/arc/training.txt') - Load all training tasks")
print("  load_arc_data('data/raw/arc/evaluation.txt') - Load all evaluation tasks") 
print("  load_random_tasks('data/raw/arc/training.txt', 10) - Load 10 random training tasks")

In [ ]:
def visualize_task_pairs(task_data, task_id, max_pairs=3):
    """Visualize input/output pairs for a specific task in 2-column format"""
    if task_id not in task_data:
        print(f"Task {task_id} not found in data")
        return
    
    task = task_data[task_id]
    train_pairs = task['train']
    test_pairs = task['test']
    
    num_train = min(len(train_pairs), max_pairs)
    num_test = min(len(test_pairs), max_pairs)
    total_pairs = num_train + num_test
    
    if total_pairs == 0:
        print("No pairs to visualize")
        return
    
    # Create subplot grid: 2 columns (Input | Output), rows for each pair
    fig, axes = plt.subplots(total_pairs, 2, figsize=(12, 4*total_pairs))
    
    # Handle single row case
    if total_pairs == 1:
        axes = axes.reshape(1, -1)
    
    # Plot training pairs
    for i in range(num_train):
        pair = train_pairs[i]
        input_grid = pair['input']
        output_grid = pair['output']
        
        plot_grid(input_grid, f"Train {i+1} - Input", axes[i, 0])
        plot_grid(output_grid, f"Train {i+1} - Output", axes[i, 1])
    
    # Plot test pairs
    for i in range(num_test):
        pair = test_pairs[i]
        input_grid = pair['input']
        output_grid = pair['output']
        
        row_idx = num_train + i
        plot_grid(input_grid, f"Test {i+1} - Input", axes[row_idx, 0])
        plot_grid(output_grid, f"Test {i+1} - Output", axes[row_idx, 1])
    
    plt.suptitle(f"ARC Task: {task_id}", fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

def browse_tasks(data, start_idx=0, num_tasks=5):
    """Browse through multiple tasks with their IDs"""
    task_ids = list(data.keys())
    end_idx = min(start_idx + num_tasks, len(task_ids))
    
    print(f"Showing tasks {start_idx+1} to {end_idx} of {len(task_ids)} total tasks:")
    print("-" * 50)
    
    for i in range(start_idx, end_idx):
        task_id = task_ids[i]
        task = data[task_id]
        num_train = len(task['train'])
        num_test = len(task['test'])
        print(f"{i+1:3d}. Task ID: {task_id} | Train: {num_train} | Test: {num_test}")
    
    print("-" * 50)
    print(f"To visualize a task, use: visualize_task_pairs(data, 'task_id')")

print("Task visualization functions defined!")

In [ ]:
import random

def visualize_task_from_data(data, task_id, max_pairs=3):
    """Visualize input/output pairs for a specific task from loaded data"""
    if not data or task_id not in data:
        print(f"Task {task_id} not found in provided data")
        return
    
    task = data[task_id]
    train_pairs = task['train']
    test_pairs = task['test']
    
    num_train = min(len(train_pairs), max_pairs)
    num_test = min(len(test_pairs), max_pairs)
    total_pairs = num_train + num_test
    
    if total_pairs == 0:
        print("No pairs to visualize")
        return
    
    # Create subplot grid: 2 columns (Input | Output), rows for each pair
    fig, axes = plt.subplots(total_pairs, 2, figsize=(12, 4*total_pairs))
    
    # Handle single row case
    if total_pairs == 1:
        axes = axes.reshape(1, -1)
    
    # Plot training pairs
    for i in range(num_train):
        pair = train_pairs[i]
        input_grid = pair['input']
        output_grid = pair['output']
        
        plot_grid(input_grid, f"Train {i+1} - Input", axes[i, 0])
        plot_grid(output_grid, f"Train {i+1} - Output", axes[i, 1])
    
    # Plot test pairs
    for i in range(num_test):
        pair = test_pairs[i]
        input_grid = pair['input']
        output_grid = pair['output']
        
        row_idx = num_train + i
        plot_grid(input_grid, f"Test {i+1} - Input", axes[row_idx, 0])
        plot_grid(output_grid, f"Test {i+1} - Output", axes[row_idx, 1])
    
    plt.suptitle(f"ARC Task: {task_id}", fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

def visualize_all_tasks_in_data(data, show_all_pairs=True, max_pairs=3):
    """
    Visualize all tasks in the provided data dictionary
    
    Args:
        data: Dictionary of ARC tasks (from load_arc_data or load_random_tasks)
        show_all_pairs: If True, shows all pairs; if False, limits pairs per task
        max_pairs: Maximum pairs to show per task when show_all_pairs=False
    """
    if not data:
        print("No data provided")
        return
    
    task_ids = list(data.keys())
    print(f"Visualizing {len(task_ids)} tasks:")
    print("=" * 60)
    
    for idx, task_id in enumerate(task_ids, 1):
        task = data[task_id]
        train_pairs = task['train']
        test_pairs = task['test']
        
        print(f"\n{idx}. Task ID: {task_id}")
        print(f"   Training pairs: {len(train_pairs)}")
        print(f"   Test pairs: {len(test_pairs)}")
        print("-" * 40)
        
        # Calculate total pairs to show
        if show_all_pairs:
            num_train = len(train_pairs)
            num_test = len(test_pairs)
        else:
            num_train = min(len(train_pairs), max_pairs)
            num_test = min(len(test_pairs), max_pairs)
        
        total_pairs = num_train + num_test
        
        if total_pairs == 0:
            print("No pairs to visualize")
            continue
        
        # Create subplot grid: 2 columns (Input | Output), rows for each pair
        fig, axes = plt.subplots(total_pairs, 2, figsize=(12, 4*total_pairs))
        
        # Handle single row case
        if total_pairs == 1:
            axes = axes.reshape(1, -1)
        
        row_idx = 0
        
        # Plot training pairs
        for i in range(num_train):
            pair = train_pairs[i]
            input_grid = pair['input']
            output_grid = pair['output']
            
            plot_grid(input_grid, f"Train {i+1} - Input", axes[row_idx, 0])
            plot_grid(output_grid, f"Train {i+1} - Output", axes[row_idx, 1])
            row_idx += 1
        
        # Plot test pairs
        for i in range(num_test):
            pair = test_pairs[i]
            input_grid = pair['input']
            output_grid = pair['output']
            
            plot_grid(input_grid, f"Test {i+1} - Input", axes[row_idx, 0])
            plot_grid(output_grid, f"Test {i+1} - Output", axes[row_idx, 1])
            row_idx += 1
        
        plt.suptitle(f"Task {idx}: {task_id}", fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        if idx < len(task_ids):
            print("\n" + "="*60)

def quick_random_visualization(txt_file_path, num_tasks=3):
    """
    Quick function to load random tasks and visualize them
    
    Args:
        txt_file_path: Path to txt file containing task IDs
        num_tasks: Number of random tasks to load and visualize
    """
    print(f"Loading {num_tasks} random tasks from {txt_file_path}...")
    random_data = load_random_tasks(txt_file_path, num_tasks)
    
    if random_data:
        print(f"Visualizing {len(random_data)} randomly selected tasks:")
        visualize_all_tasks_in_data(random_data, show_all_pairs=True)
    else:
        print("Failed to load random tasks")

print("Task visualization functions defined!")
print("New functions available:")
print("  visualize_task_from_data(data, 'task_id', max_pairs=3)")
print("  visualize_all_tasks_in_data(data, show_all_pairs=True)")
print("  quick_random_visualization('data/raw/arc/training.txt', num_tasks=3)")

In [ ]:
# Example 1: Load all training data
# print("Loading ALL training data...")
# training_data = load_arc_data('data/raw/arc/training.txt')

# if training_data:
#     print(f"Successfully loaded {len(training_data)} training tasks")
    
#     # Show some example task IDs and their structure
#     sample_task_ids = list(training_data.keys())[:3]
#     print(f"Sample task IDs: {sample_task_ids}")
    
#     # Show structure of first task
#     if sample_task_ids:
#         first_task = training_data[sample_task_ids[0]]
#         print(f"Structure of task '{sample_task_ids[0]}':")
#         print(f"   Training examples: {len(first_task['train'])}")
#         print(f"   Test examples: {len(first_task['test'])}")
        
#         if first_task['train']:
#             example = first_task['train'][0]
#             input_grid = example['input']
#             output_grid = example['output']
#             print(f"   First training example:")
#             print(f"     Input size: {len(input_grid)}x{len(input_grid[0])}")
#             print(f"     Output size: {len(output_grid)}x{len(output_grid[0])}")
# else:
#     print("Failed to load training data")

print("\n" + "="*50)

# Example 2: Load random subset of training data
print("Loading RANDOM SUBSET of training data...")
random_training_data = load_random_tasks('data/raw/arc/training.txt', num_tasks=3)

if random_training_data:
    print(f"Successfully loaded {len(random_training_data)} random training tasks")
    print(f"Random task IDs: {list(random_training_data.keys())}")
else:
    print("Failed to load random training data")

In [ ]:
# Visualization Examples - Uncomment to use

# Option 1: Visualize a specific task from loaded data
# if training_data:
#     first_task_id = list(training_data.keys())[0]
#     visualize_task_from_data(training_data, first_task_id, max_pairs=2)

# Option 2: Visualize all tasks in random subset
if random_training_data:
    visualize_all_tasks_in_data(random_training_data, show_all_pairs=True)

# Option 3: Quick random visualization (loads and shows in one step)
# quick_random_visualization('data/raw/arc/training.txt', num_tasks=3)

# Option 4: Load and visualize random evaluation tasks
# random_eval_data = load_random_tasks('data/raw/arc/evaluation.txt', num_tasks=2)
# if random_eval_data:
#     visualize_all_tasks_in_data(random_eval_data, show_all_pairs=False, max_pairs=2)

print("Visualization options available:")
print("1. visualize_task_from_data(data, 'task_id', max_pairs=N) - specific task from loaded data")
print("2. visualize_all_tasks_in_data(data, show_all_pairs=True) - all tasks in data dict")
print("3. quick_random_visualization('path/to/file.txt', num_tasks=N) - load and show random tasks")
print("4. load_random_tasks('path/to/file.txt', N) - load N random tasks into data dict")
print("\nTo run examples, uncomment the code above and execute the cell.")

In [ ]:
# Load evaluation data for comparison
# eval_data = load_arc_data('data/raw/arc/evaluation.txt')

# if eval_data:
#     print(f"Successfully loaded {len(eval_data)} evaluation tasks")
    
#     # Show some example task IDs
#     sample_eval_ids = list(eval_data.keys())[:3]
#     print(f"Sample evaluation task IDs: {sample_eval_ids}")
    
#     # Compare with training data
#     if training_data:
#         print(f"Dataset Comparison:")
#         print(f"   Training tasks: {len(training_data)}")
#         print(f"   Evaluation tasks: {len(eval_data)}")
        
#         # Check for any overlap (should be none)
#         overlap = set(training_data.keys()) & set(eval_data.keys())
#         if overlap:
#             print(f"   Overlap found: {len(overlap)} tasks")
#         else:
#             print(f"   No overlap - clean data split")
# else:
#     print("Failed to load evaluation data")

# print("\n" + "="*50)

# Example: Load random evaluation tasks
print("Loading RANDOM SUBSET of evaluation data...")
random_eval_data = load_random_tasks('data/raw/arc/evaluation.txt', num_tasks=3)

if random_eval_data:
    print(f"Successfully loaded {len(random_eval_data)} random evaluation tasks")
    print(f"Random evaluation task IDs: {list(random_eval_data.keys())}")
else:
    print("Failed to load random evaluation data")

## Quick Start Guide

1. **Setup**: Run the first two cells to install dependencies and import libraries
2. **Load Data**: Execute the data loading cells to see available tasks
3. **Browse**: Use `browse_tasks(data, start_idx=0, num_tasks=10)` to see task IDs
4. **Visualize**: Choose from multiple visualization options below
5. **Explore**: Try different parameters to understand various puzzle patterns

## Visualization Options

### Random Task Selection (NEW!)
- `visualize_random_sample()`: Quick way to see 3 random tasks from loaded data
- `visualize_random_tasks(data, num_tasks=3, show_all_pairs=True)`: Custom random selection

### Specific Task Selection
- `visualize_task_pairs(data, 'task_id', max_pairs=3)`: Show specific task with limited pairs
- `browse_tasks(data, start_idx=0, num_tasks=10)`: List available task IDs

## Key Functions

- `load_arc_data(path)`: Load training or evaluation data
- `browse_tasks(data, start_idx, num_tasks)`: List available tasks with IDs
- `visualize_task_pairs(data, task_id, max_pairs)`: Show input/output pairs for specific task
- `visualize_random_tasks(data, num_tasks, show_all_pairs)`: Show random tasks with all/limited pairs
- `visualize_random_sample()`: Quick random selection from loaded data
- `plot_grid(grid, title)`: Plot individual grid with proper colors

## Parameters Explained

- `max_pairs`: Limits pairs shown per task (for specific task visualization)
- `num_tasks`: Number of tasks to display (for browsing or random selection)
- `show_all_pairs`: If True, shows ALL pairs in selected tasks; if False, limits to 3 pairs
- `start_idx`: Starting position for browsing tasks

## Tips

- Each task has multiple training pairs and test pairs
- Grids use colors 0-9 mapped to distinct colors
- Input/Output pairs show the transformation pattern to learn
- Random selection helps discover diverse puzzle types
- Use `show_all_pairs=False` for large tasks to avoid overwhelming displays